<a href="https://colab.research.google.com/github/philipplukas/InvestigatingEmergence/blob/main/PretrainingTransformerXL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
%cd /content/drive/My Drive/

/content/drive/My Drive


In [63]:
!rm -dr InvestigatingEmergence

In [64]:
!git clone --recursive https://github.com/philipplukas/InvestigatingEmergence

Cloning into 'InvestigatingEmergence'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 15 (delta 1), reused 11 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), 3.48 KiB | 25.00 KiB/s, done.
Submodule 'transformer-xl' (https://github.com/kimiyoung/transformer-xl.git) registered for path 'transformer-xl'
Cloning into '/content/drive/My Drive/InvestigatingEmergence/transformer-xl'...
remote: Enumerating objects: 164, done.        
remote: Total 164 (delta 0), reused 0 (delta 0), pack-reused 164        
Receiving objects: 100% (164/164), 113.49 KiB | 5.16 MiB/s, done.
Resolving deltas: 100% (98/98), done.
Submodule path 'transformer-xl': checked out '44781ed21dbaec88b280f74d9ae2877f52b492a5'


In [20]:
!git pull origin master

From https://github.com/kimiyoung/transformer-xl
 * branch            master     -> FETCH_HEAD
Already up to date.


In [2]:
%cd "/content/drive/My Drive/InvestigatingEmergence"

/content/drive/My Drive/InvestigatingEmergence


In [3]:
%cd "transformer-xl"

/content/drive/My Drive/InvestigatingEmergence/transformer-xl


In [67]:
!ls

getdata.sh  LICENSE  prep_text8.py  pytorch  README.md	tf


In [68]:
!chmod u+x getdata.sh && ./getdata.sh

=== Acquiring datasets ===
---
- Downloading WikiText-2 (WT2)
^C


In [69]:
%cd pytorch

/content/drive/My Drive/InvestigatingEmergence/transformer-xl/pytorch


In [70]:
!chmod u+x run_enwik8_base.sh

In [ ]:
!./run_enwik8_base.sh train

In [ ]:
""" ORIGNAL parameters

python train.py \
        --cuda \
        --data ../data/enwik8/ \
        --dataset enwik8 \
        --n_layer 12 \
        --d_model 512 \
        --n_head 8 \
        --d_head 64 \
        --d_inner 2048 \
        --dropout 0.1 \
        --dropatt 0.0 \
        --optim adam \
        --lr 0.00025 \
        --warmup_step 0 \
        --max_step 400000 \
        --tgt_len 512 \
        --mem_len 512 \
        --eval_tgt_len 128 \
        --batch_size 22 \
"""

In [ ]:
!python train.py \
        --cuda \
        --data ../data/ctl/ \
        --dataset ctl \
        --n_layer 4 \
        --d_model 128 \
        --n_head 2 \
        --d_head 16 \
        --d_inner 256 \
        --dropout 0.1 \
        --dropatt 0.0 \
        --optim adam \
        --lr 0.00025 \
        --warmup_step 0 \
        --max_step 400000 \
        --tgt_len 512 \
        --mem_len 512 \
        --eval_tgt_len 128 \
        --batch_size 22

Experiment dir : LM-TFM-enwik8/20230301-130050
Loading cached dataset...
    - data : ../data/enwik8/
    - dataset : enwik8
    - n_layer : 4
    - n_head : 2
    - d_head : 16
    - d_embed : 128
    - d_model : 128
    - d_inner : 256
    - dropout : 0.1
    - dropatt : 0.0
    - init : normal
    - emb_init : normal
    - init_range : 0.1
    - emb_init_range : 0.01
    - init_std : 0.02
    - proj_init_std : 0.01
    - optim : adam
    - lr : 0.00025
    - mom : 0.0
    - scheduler : cosine
    - warmup_step : 0
    - decay_rate : 0.5
    - lr_min : 0.0
    - clip : 0.25
    - clip_nonemb : False
    - max_step : 400000
    - batch_size : 22
    - batch_chunk : 1
    - tgt_len : 512
    - eval_tgt_len : 128
    - ext_len : 0
    - mem_len : 512
    - not_tied : False
    - seed : 1111
    - cuda : True
    - adaptive : False
    - div_val : 1
    - pre_lnorm : False
    - varlen : False
    - multi_gpu : False
    - log_interval : 200
    - eval_interval : 4000
    - work_dir : LM

## Pretraining on CTL task

In [7]:
%cd /content/drive/MyDrive/InvestigatingEmergence

/content/drive/MyDrive/InvestigatingEmergence


In [8]:
from generate_task import CTLTask

In [9]:
import random
random.seed(0)

In [10]:
task = CTLTask()
task.max_depth = 1
base_tasks = task.generate_tasks()
iterator = task.infinite_samples(base_tasks)

In [12]:
!mkdir data %% mkdir data/ctl

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘%%’: File exists
mkdir: cannot create directory ‘mkdir’: File exists


In [11]:
with open("data/ctl/train.txt", 'w') as fp:
  for i in range(100000):
    fp.write(next(iterator) + "\n")

In [12]:
with open("data/ctl/valid.txt", 'w') as fp:
  for i in range(2000):
    fp.write(next(iterator) + "\n")

In [13]:
with open("data/ctl/test.txt", 'w') as fp:
  for i in range(2000):
    fp.write(next(iterator) + "\n")

In [5]:
%cd transformer-xl/pytorch

[Errno 2] No such file or directory: 'transformer-xl/pytorch'
/content/drive/My Drive/InvestigatingEmergence/transformer-xl


In [21]:
!python train.py \
        --cuda \
        --data ../../data/ctl/ \
        --dataset ctl \
        --n_layer 4 \
        --d_model 128 \
        --n_head 4 \
        --d_head 32 \
        --d_inner 512 \
        --dropout 0.1 \
        --dropatt 0.0 \
        --optim adam \
        --lr 2e-05 \
        --warmup_step 0 \
        --max_step 400000 \
        --tgt_len 512 \
        --scheduler constant \
        --mem_len 0 \
        --ext_len 0 \
        --batch_size 22 \
        --not_tied \
        --eval-interval 1000

Experiment dir : LM-TFM-ctl/20230307-110317
Loading cached dataset...
    - data : ../../data/ctl/
    - dataset : ctl
    - n_layer : 4
    - n_head : 4
    - d_head : 32
    - d_embed : 128
    - d_model : 128
    - d_inner : 512
    - dropout : 0.1
    - dropatt : 0.0
    - init : normal
    - emb_init : normal
    - init_range : 0.1
    - emb_init_range : 0.01
    - init_std : 0.02
    - proj_init_std : 0.01
    - optim : adam
    - lr : 2e-05
    - mom : 0.0
    - scheduler : constant
    - warmup_step : 0
    - decay_rate : 0.5
    - lr_min : 0.0
    - clip : 0.25
    - clip_nonemb : False
    - max_step : 400000
    - batch_size : 22
    - batch_chunk : 1
    - tgt_len : 512
    - eval_tgt_len : 50
    - ext_len : 0
    - mem_len : 0
    - not_tied : True
    - seed : 1111
    - cuda : True
    - adaptive : False
    - div_val : 1
    - pre_lnorm : False
    - varlen : False
    - multi_gpu : False
    - log_interval : 200
    - eval_interval : 1000
    - work_dir : LM-TFM-ctl/2

## Evaluation of model

In [22]:
%cd pytorch

[Errno 2] No such file or directory: 'pytorch'
/content/drive/My Drive/InvestigatingEmergence/transformer-xl/pytorch


In [7]:
!ls

data  getdata.sh  LICENSE  prep_text8.py  pytorch  README.md  tf


In [23]:
from mem_transformer import MemTransformerLM
from data_utils import get_lm_corpus


In [24]:
import torch

In [26]:
model = torch.load("/content/drive/MyDrive/InvestigatingEmergence/transformer-xl/pytorch/LM-TFM-ctl/20230307-110317/model.pt")
model.eval()

MemTransformerLM(
  (word_emb): AdaptiveEmbedding(
    (emb_layers): ModuleList(
      (0): Embedding(24, 128)
    )
    (emb_projs): ParameterList()
  )
  (drop): Dropout(p=0.1, inplace=False)
  (layers): ModuleList(
    (0): RelPartialLearnableDecoderLayer(
      (dec_attn): RelPartialLearnableMultiHeadAttn(
        (qkv_net): Linear(in_features=128, out_features=384, bias=False)
        (drop): Dropout(p=0.1, inplace=False)
        (dropatt): Dropout(p=0.0, inplace=False)
        (o_net): Linear(in_features=128, out_features=128, bias=False)
        (layer_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (r_net): Linear(in_features=128, out_features=128, bias=False)
      )
      (pos_ff): PositionwiseFF(
        (CoreNet): Sequential(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): ReLU(inplace=True)
          (2): Dropout(p=0.1, inplace=False)
          (3): Linear(in_features=512, out_features=128, bias=True)
          (4): D

In [27]:
test_input = "PCe:011."

In [21]:
del get_lm_corpus

In [28]:
corpus = get_lm_corpus('/content/drive/MyDrive/InvestigatingEmergence/data/ctl', 'ctl')

Loading cached dataset...


In [29]:
vocab = corpus.vocab

In [30]:
tokenized = vocab.tokenize(test_input)

In [31]:
tokenized

['P', 'C', 'e', ':', '0', '1', '1', '.']

In [32]:
encoded = vocab.encode_sents(tokenized)

In [33]:
encoded = encoded[0].to(device='cuda')

In [34]:
hidden = model._forward(encoded[0].reshape(1,-1))[0]

In [35]:
hidden

tensor([[[ 0.8713,  0.6302, -1.6254,  0.7747,  1.0746,  0.8525, -2.1163,
           0.8879, -0.2778,  0.7046,  0.8404,  0.2602, -1.1572, -1.1093,
          -0.6335,  1.3267,  0.6690,  0.6009, -0.9030,  1.3282, -1.0425,
           0.6712,  0.4485,  0.8645,  0.4713, -0.3256, -0.5740,  1.1265,
          -0.4830, -0.2819, -1.5795, -0.6691, -1.3927, -0.7132,  0.8412,
          -0.9770,  1.5757, -0.5886, -0.9270,  0.8101,  1.3465,  1.2275,
          -0.9836, -1.0858, -1.3419,  0.1219, -1.1723, -0.9212, -0.6683,
           0.8550, -0.7383, -0.7051,  1.1626,  0.1884, -1.0734,  0.5911,
           0.8621, -1.0743,  1.1788,  0.8624,  0.7972, -0.5683, -0.8697,
          -0.9260,  1.1128,  2.3606,  0.7314,  1.0388, -1.1307,  1.3042,
          -1.4518,  1.3202,  0.7782, -1.5742,  0.9142,  1.0308, -0.9864,
          -1.3306, -0.9901, -1.1946,  0.9400, -0.8331,  1.6735, -0.5811,
           1.0313,  0.8003, -0.9446,  0.5132, -0.8145, -1.0032, -0.5406,
          -1.1596, -0.8925, -1.1434, -1.1523,  0.96

In [36]:
logit = model.crit._compute_logit(hidden, model.crit.out_layers[0].weight,
                                        model.crit.out_layers[0].bias, model.crit.out_projs[0])

In [37]:
logit

tensor([[[-0.1087, -0.1563,  0.1239,  0.0492,  0.3566, -0.1502, -0.2168,
          -0.0235, -0.7082,  6.2037, -1.0107, -0.4975,  0.2330, -0.8392,
          -0.6418,  0.1465,  0.1044, -0.0270,  0.1135, -0.4286,  0.0449,
          -0.2254, -0.1221, -0.2554]]], device='cuda:0',
       grad_fn=<ViewBackward0>)

In [17]:
vocab.sym2idx

OrderedDict([('a', 0),
             ('b', 1),
             ('c', 2),
             ('d', 3),
             ('e', 4),
             ('f', 5),
             ('g', 6),
             ('h', 7),
             ('N', 8),
             ('C', 9),
             ('P', 10),
             ('.', 11),
             (':', 12),
             ('0', 13),
             ('1', 14),
             ('2', 15),
             ('3', 16),
             ('4', 17),
             ('5', 18),
             ('6', 19),
             ('7', 20),
             ('8', 21),
             ('9', 22),
             ('<UNK>', 23)])